In [ ]:
import sys, os, subprocess, shutil, glob, pathlib

print("Python:", sys.version)
print("Interpreter:", sys.executable)

env_name = os.path.basename(sys.prefix) or "current-env"
print("Detected environment name:", env_name)

req_file = "requirements.txt"
if os.path.exists(req_file):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", req_file])
else:
    subprocess.check_call([sys.executable, "-m", "pip", "install",
                           "numpy==1.23.5", "scipy==1.15.3",
                           "parmap==1.7.0", "pandas==2.3.1",
                           "scikit-learn==1.7.1"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "setuptools>=68", "wheel", "Cython<3.1"])
subprocess.check_call([sys.executable, "-m", "ipykernel", "install",
                       "--user", "--name", env_name, "--display-name", f"Python ({env_name})"])

for p in ["build", "__pycache__"]:
    shutil.rmtree(p, ignore_errors=True)
for f in glob.glob("hmm_bw.*.so") + glob.glob("hmm_bw.*.pyd") + glob.glob("hmm_bw.c"):
    pathlib.Path(f).unlink(missing_ok=True)

for key in ("CFLAGS", "CPPFLAGS", "CXXFLAGS"):
    if key in os.environ:
        os.environ.pop(key)
        
if os.path.exists("setup2.py"):
    print("\n[Build] python setup.py build_ext --inplace ...")
    subprocess.check_call([sys.executable, "setup2.py", "build_ext", "--inplace"])
else:
    raise FileNotFoundError("setup2.py not found in current directory.")

import importlib
try:
    import hmm_bw
except Exception:
    if "hmm_bw" in sys.modules:
        del sys.modules["hmm_bw"]
    importlib.invalidate_caches()
    import hmm_bw

print("\nSetup complete. hmm_bw loaded:", hmm_bw)
print(f"Jupyter kernel registered as: Python ({env_name})")

In [ ]:
import numpy as np
import scipy as sp
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import gridspec
import pandas as pd
import sys, site, platform
import pickle

print("Python:", sys.version)
print("Interpreter:", sys.executable)
print("Site-packages:", site.getsitepackages())
print("OS:", platform.platform())

In [ ]:
# user-configurable parameters
user_params = {
    "D_min": 0.001,
    "D_max": 2.0,
    "unit_T": 0.1,
    "unit_L": 0.167,
}
path = 'test_dataset'

hmm_bw.set_params(user_params)
fnames = np.genfromtxt(f'{path}/filenames.txt', dtype=str)
print(hmm_bw.get_params())  # check

fnum = len(fnames)
findices = np.arange(1, fnum + 1)
num_cores = min(os.cpu_count(), fnum)
indices = np.array_split(findices, num_cores)

print(fnames)
for i in range(fnum):
    globals()[f'trj_{i+1}'] = np.genfromtxt(f'{path}/{fnames[i]}')

In [ ]:
def load_top3_means(path):
    with open(path, "rb") as f:
        obj = pickle.load(f)

    arrs = obj.get("arrays", {})
    def _get(name):
        a = np.asarray(arrs.get(name, [np.nan, np.nan, np.nan]), float).ravel()
        if a.size < 3:
            a = np.pad(a, (0, 3 - a.size), constant_values=np.nan)
        a = np.sort(a[:3])
        return a

    return {
        "d_aicmle": _get("d_aicmle"),
        "d_aicmap": _get("d_aicmap"),
        "d_baymle": _get("d_baymle"),
        "d_baymap": _get("d_baymap"),
    }

loaded = load_top3_means("bgmm_top3_means.pkl")
d_aicmle = loaded["d_aicmle"]
d_aicmap = loaded["d_aicmap"]
d_baymle = loaded["d_baymle"]
d_baymap = loaded["d_baymap"]

unit_T = 0.1

cands = [arr for arr in [d_aicmle, d_aicmap, d_baymle, d_baymap]
         if arr is not None and np.isfinite(arr).any()]

if len(cands) == 0:
    d_avg = np.array([np.nan, np.nan, np.nan], dtype=float)
else:
    stacked = np.vstack(cands)          # (m, 3)
    d_avg = np.nanmean(stacked, axis=0)

sigma = np.sqrt(2.0 * d_avg * unit_T)[:, None]  # (3,1)
widths = np.sqrt(2.0 * d_avg * unit_T)          # (3,)

print("d_aicmle:", d_aicmle)
print("d_aicmap:", d_aicmap)
print("d_baymle:", d_baymle)
print("d_baymap:", d_baymap)
print("d_avg:", d_avg)
print("sigma:", sigma.ravel())
print("widths:", widths)


In [ ]:
#data

path = 'test_dataset'

fnames = np.genfromtxt(f'{path}/filenames.txt', dtype=str)
fnum = len(fnames)

for i in range(fnum):
    globals()[f'trj_{i+1}'] = np.genfromtxt(f'{path}/{fnames[i]}')
        
obs_v = []
for i in range(fnum):
    obs = globals()[f'trj_{i+1}'].copy()
    obs_v.append(np.diff(obs)[:, None])
obs_v = np.concatenate(obs_v).ravel()

# Transition matrix estimation

In [ ]:
obs_v = obs_v[:, None]
P = hmm_bw.baum_skel_for_P(obs_v, d_avg, nstates = 3, iteration = 4, stop_logratio = 0.0001, max_cnt = 100)

# trajectory segmentation

In [ ]:
#showcase = [0, 24]
colors = ['k', 'tab:blue', 'tab:red']

for i in range(fnum):
    print('i:', i)
    fig = plt.figure(figsize=(5, 2.5))
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
    gs = gridspec.GridSpec(nrows=2,
                           ncols=1,
                           height_ratios=[1, 1],
                           width_ratios=[1])    
    
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1], sharex = ax0)
    
    obs = globals()[f'trj_{i+1}'].copy()
    time = np.arange(len(obs)) * 0.1
    obs_v = np.diff(obs)[:, None]
    est_x = np.argmax(hmm_bw.gamma_est(obs_v, P, sigma, 3), axis = 0)
    
    # shade
    ax1.fill_between(time[:len(obs)-1], -widths[1], widths[1], color = 'tab:blue', alpha = 0.2)
    ax1.fill_between(time[:len(obs)-1], -widths[2], -widths[1], color = 'tab:red', alpha = 0.2)
    ax1.fill_between(time[:len(obs)-1], widths[1], widths[2], color = 'tab:red', alpha = 0.2)
    #ax1.fill_between(time[:len(obs)-1], -widths[0], widths[0], where=(est_x == 0), color = 'tab:blue', alpha = 0.2)
    #ax1.fill_between(time[:len(obs)-1], -widths[1], widths[1], where=(est_x == 1), color = 'tab:red', alpha = 0.2)
    # shade
    
    ax1.plot(time[:len(time) - 1], obs_v, c = 'grey', alpha = 0.3)
    for j in range(3):
        ax1.scatter(time[:len(time) - 1][est_x == j], obs_v[est_x == j], c = colors[j], s = 5, label = 'state {}'.format(j+1))

    ax1.set_xlabel(r'time ($\mathrm{s}$)', fontsize = 15)
    ax1.set_ylabel(r'$\Delta x$ ($\mathrm{\mu m}$)', fontsize = 15)
    ax1.tick_params(which = 'both', labelsize = 12, direction = 'in')
    ax1.set_ylim(-1, 1)
    ax1.yaxis.set_label_coords(-0.06, 0.5)
    ax1.legend(loc = 'center right', bbox_to_anchor = (1.3, 1.15))
    
    est_x = np.append(est_x, np.array([-1]))
    
    ax0.plot(time, obs, c = 'grey', alpha = 0.3)
    for j in range(3):
        ax0.scatter(time[est_x == j], obs[est_x == j], c = colors[j], s = 5)
    
#    ax0.set_xlabel(r'time ($\mathrm{s}$)', fontsize = 15)
    ax0.set_ylabel(r'$x$ ($\mathrm{\mu m}$)', fontsize = 15)
    ax0.yaxis.set_label_coords(-0.06, 0.5)
    ax0.tick_params(which = 'both', labelsize = 12, direction = 'in')
    
    #ax0.set_xlim(0, 15)
    
    #plt.savefig('segment{}.pdf'.format(i+1), format = 'pdf', dpi = 1200)
    plt.show()